# Naive Bayes

A probabilistic classifier.
Given $n$ _attributes_, that is

$$
\text{attributes} = \lbrace{a_1, a_2, a_3, \dots, a_n \rbrace},
$$

and a set of $k$ _labels_, we can calculate the conditional probability of each label given a certain set of attributes. That is:

$$
p\left(L_k|a_1, a_2, a_3,\dots,a_n\right)
$$.

This is done by direct application of Baye's theorem:

$$
p\left(L_k|a_1,a_2,a_3,\dots,a_n\right) = \frac{p\left(L_k\right)p\left(a_1,a_2,a_3,\dots,a_n|L_k\right)}{p\left(a_1,a_2,a_3,\dots,a_n\right)}
$$

## Example

For the following minimal example a Java implementation of the Naive Bayes algorithm will be used (available at https://github.com/ruivieira/java-naive-bayes).

Consider a simple IT purchasing system where the user can choose a laptop brand (`Apple` or `Lenovo`) for use in a specific department (`design` or `accounting`) in one of two offices (`US` or `UK`).

In this case, NB will try to classify the laptop brand according to the historical purchase data. It is clear then that the attributes will be:

$$
    a = \lbrace\text{user}, \text{department}, \text{office}\rbrace
$$

and the labels will be

$$
    L = \lbrace\text{Brand A}, \text{Brand B}\rbrace
$$

In [1]:
%maven org.ruivieira:naivebayes:0.1-SNAPSHOT

In [2]:
import org.ruivieira.ml.naivebayes.NaiveBayes;
import org.ruivieira.ml.naivebayes.Model;

import java.util.Map;

In [3]:
Model model = Model.create();

Let's start by adding the first record. User Anna buys an Apple for the US design department.

In [4]:
model.train(new String[]{"Anna", "design", "US"}, "Apple");

If we try to predict that label (outcome) for any of the attributes, the result will be unsurprising:

In [5]:
NaiveBayes naiveBayes = new NaiveBayes(model);
System.out.println("Anna: " + naiveBayes.classify(new String[]{"Anna"}).toString());
System.out.println("design: " + naiveBayes.classify(new String[]{"design"}).toString());
System.out.println("US: " + naiveBayes.classify(new String[]{"US"}).toString());

Anna: {Apple=100.0}
design: {Apple=100.0}
US: {Apple=100.0}


We now add a purchase for a Lenovo for US accounting department.

In [6]:
model.train(new String[]{"Anna", "accounting", "US"}, "Lenovo");
naiveBayes = new NaiveBayes(model);
System.out.println("Anna: " + naiveBayes.classify(new String[]{"Anna"}).toString());
System.out.println("design: " + naiveBayes.classify(new String[]{"design"}).toString());
System.out.println("US: " + naiveBayes.classify(new String[]{"US"}).toString());

Anna: {Lenovo=50.0, Apple=50.0}
design: {Lenovo=5.0E-9, Apple=50.0}
US: {Lenovo=50.0, Apple=50.0}


Nothing we couldn't figure out ourselves, yet. Anna is 50/50 as likely to buy a Lenovo or an Apple. The design department is more likely to get an Apple (50% vs. ~0%) and the US office is as likely to get a Lenovo or an Apple.

Let's now add a third user, Bill. Bill makes the same purchasing choices as Anna, but he also buys for the UK office.

In [7]:
model.train(new String[]{"Bill", "accounting", "US"}, "Lenovo");
model.train(new String[]{"Bill", "design", "US"}, "Apple");
model.train(new String[]{"Bill", "accounting", "UK"}, "Lenovo");
model.train(new String[]{"Bill", "design", "UK"}, "Apple");

In [8]:
naiveBayes = new NaiveBayes(model);
System.out.println("Anna: " + naiveBayes.classify(new String[]{"Anna"}).toString());
System.out.println("Bill: " + naiveBayes.classify(new String[]{"Anna"}).toString());
System.out.println("design: " + naiveBayes.classify(new String[]{"design"}).toString());
System.out.println("US: " + naiveBayes.classify(new String[]{"US"}).toString());

Anna: {Lenovo=16.666666666666664, Apple=16.666666666666664}
Bill: {Lenovo=16.666666666666664, Apple=16.666666666666664}
design: {Lenovo=5.0E-9, Apple=50.0}
US: {Lenovo=33.33333333333333, Apple=33.33333333333333}


Still nothing surprising. However, one of the strengths of NB is the ability to combine attributes to get insights. Let's see that adding another user, Claire. Claire will buy Lenovos for all the offices and departments.

In [9]:
model.train(new String[]{"Claire", "accounting", "US"}, "Lenovo");
model.train(new String[]{"Claire", "design", "US"}, "Lenovo");
model.train(new String[]{"Claire", "accounting", "UK"}, "Lenovo");
model.train(new String[]{"Claire", "design", "UK"}, "Lenovo");

In [10]:
naiveBayes = new NaiveBayes(model);
System.out.println("Claire: " + naiveBayes.classify(new String[]{"Claire"}).toString());
System.out.println("design: " + naiveBayes.classify(new String[]{"design"}).toString());
System.out.println("design US: " + naiveBayes.classify(new String[]{"design", "US"}).toString());
System.out.println("Bill accounting: " + naiveBayes.classify(new String[]{"Bill accounting"}).toString());

Claire: {Lenovo=40.0, Apple=3.0E-9}
design: {Lenovo=20.0, Apple=30.0}
design US: {Lenovo=11.428571428571427, Apple=20.0}
Bill accounting: {Lenovo=70.0, Apple=30.0}


Here we start to see the usefulness of combining attributes.